In [1]:
from transliterate import translit

import pandas as pd
import numpy as np

import json
import re
import os

# TODO
* MVP with only personal chats
* Add aliases to LINKS and PICTURES (a lot of data is based on that)
* Add aliases to emojis

In [2]:
BASIC_PATH = './'

In [3]:
# with open(f'{BASIC_PATH}/data/result.json', 'r') as inp, open(f'{BASIC_PATH}/data/chats.json', 'w') as out:
#     doc = json.load(inp)
#     out.write(json.dumps(doc['chats']['list']))

In [ ]:
data = pd.read_json(f'{BASIC_PATH}/data/chats.json')
data

In [5]:
FILTER_CHATS = ['Telegram']

In [ ]:
personal_chats = data[(data['type'] == 'personal_chat') & (~data['name'].isin(FILTER_CHATS))][['name', 'id']]
personal_chats

In [ ]:
group_chats = data[data['type'].isin(['private_supergroup', 'private_group']) ][['name', 'id']]
group_chats

In [8]:
ME_ID = 'user372675274'

In [9]:
personal_chats.shape, group_chats.shape

((159, 2), (106, 2))

In [ ]:
messages = pd.DataFrame(data[data['name'] ==  'Ангелина 🦊']['messages'][2])
messages['date_unixtime'] = messages['date_unixtime'].apply(int)
messages

In [11]:
def shrink(messages):
    messages_shrinked = pd.DataFrame()
    i = -1
    for idx, row in messages.iterrows():
        if len(messages_shrinked) > 0 and messages_shrinked.loc[i, 'from_id'] == row['from_id'] and row['date_unixtime'] - messages_shrinked.loc[i, 'date_unixtime'] <= 5 * 60:
            txt = row['text']
            if type(row['text']) == list:
                txt = ''
                for item in row['text']:
                    if type(item) == dict and 'type' in item and item['type'] in ['link']:
                        txt += f'<{item["type"].upper()}>'
                    elif type(item) == dict and 'text' in item:
                        txt += item['text']
                    else:
                        txt += item
            messages_shrinked.loc[i, 'text'] = messages_shrinked.loc[i, 'text'] + '\n' + txt
        else:
            new_row = pd.DataFrame(row).T
            messages_shrinked = pd.concat([messages_shrinked, new_row], axis=0, ignore_index=True).reset_index(drop=True)
            i+=1
            txt = row['text']
            if type(row['text']) == list:
                txt = ''
                for item in row['text']:
                    if type(item) == dict and 'type' in item and item['type'] in ['link']:
                        txt += f'<{item["type"].upper()}>'
                    elif type(item) == dict and 'text' in item:
                        txt += item['text']
                    else:
                        txt += item
            messages_shrinked.loc[i, 'text'] = txt
    return messages_shrinked

In [12]:
def get_messages(data, agent):
    messages = pd.DataFrame(data[data['id'] ==  agent]['messages'].iloc[0])
    messages = messages[messages['type'] == 'message']
    messages = messages[messages['text'] != '']
    messages['date_unixtime'] = messages['date_unixtime'].apply(int)
    return messages

In [16]:
def build_dialog(messages):
    """
    * First message should always be not from me.
    * If some message was 'ignored' - then add answer as empty string
    * Dialog is considered finished it the last message was more than 8 hours ago
    """
    columns = ['id', 'date', 'date_unixtime', 'from', 'from_id', 'prompt', 'answer']
    dialog = pd.DataFrame(columns=columns)
    
    i = -1
    for idx, row in messages.iterrows():
        answer = row['from_id'] == ME_ID
        if len(dialog) == 0 or answer and dialog.loc[i, 'date_unixtime'] - row['date_unixtime'] >= 8 * 60 * 60:
            dialog = pd.concat([dialog, pd.DataFrame({'prompt': '', 'date_unixtime': row['date_unixtime']}, index=[1])], axis=0, ignore_index=True)
            i+=1
        elif not answer:
            new_row = pd.DataFrame({
                'id': row['id'],
                'date': row['date'],
                'date_unixtime': row['date_unixtime'],
                'from': row['from'],
                'from_id': row['from_id'],
                'prompt': row['text']
            }, index=[1])
            dialog = pd.concat([dialog, new_row], axis=0, ignore_index=True)
            i+=1

        # less than 8 hours passed between messages
        if answer and dialog.loc[i, 'date_unixtime'] - row['date_unixtime'] < 8 * 60 * 60:
            dialog.loc[i, 'answer'] = row['text']
        else:
            dialog.loc[i, 'answer'] = np.nan
    return dialog

In [ ]:
for idx, row in personal_chats.loc[:, :].iterrows():
    print('Building data for:', idx, row['name'])
    messages = shrink(get_messages(data, row['id']))
    if len(messages) == 0:
        continue
    dialog = build_dialog(messages)
    raw_name = row['name']
    if row['name'] is None:
        raw_name = 'DEFAULT'
    name = re.sub(r'\W+', '', raw_name)
    
    i = 0
    while os.path.exists(f'{BASIC_PATH}/data/messages/{translit(name, "ru", reversed=True) + str(i)}.csv'):
        i+=1
    
    print('Saving... <', f'{BASIC_PATH}/data/messages/{translit(name, "ru", reversed=True) + str(i)}.csv', sep='=' * 10)
    dialog.to_csv(f'{BASIC_PATH}/data/messages/{translit(name, "ru", reversed=True) + str(i)}.csv')

In [ ]:
os.listdir(f'{BASIC_PATH}/data/messages')

In [ ]:
df = pd.DataFrame()
for filename in os.listdir(f'{BASIC_PATH}/data/messages'):
    df = pd.concat([df, pd.read_csv(f'{BASIC_PATH}/data/messages/{filename}', encoding = "utf-8")], axis=0, ).drop(columns=['Unnamed: 0'])
df

In [ ]:
df = df[~df['answer'].isin([np.nan, None, '']) & ~df['prompt'].isin([np.nan, None, ''])]
df

In [37]:
df.to_csv(f'{BASIC_PATH}/data/messages/full.csv', index=False)